In [3]:
import numpy as np
import json
from glob import glob
import os
import random
import cv2

## 1. 데이터 로드

In [16]:
sorted_pose_files = sorted(glob("/home/ros/llm_robot/data/Calibration/Eye-in-Hand2/poses/*.json"))
sorted_charuco_files = sorted(glob("/home/ros/llm_robot/data/Calibration/Eye-in-Hand2/*.json"))

In [17]:
len(sorted_pose_files), len(sorted_charuco_files)

(20, 21)

## 2. Pose, Charuco 로드 및 변환

In [18]:
R_gripper2base_all = []
t_gripper2base_all = []
R_target2cam_all = []
t_target2cam_all = []

for pf, cf in zip(sorted_pose_files, sorted_charuco_files[:-1]):
    with open(pf, "r") as f:
        pose = json.load(f)
    with open(cf, "r") as f:
        charuco = json.load(f)

    R_base2ee = np.array(pose["R_base2ee"])
    t_base2ee = np.array(pose["t_base2ee"]).reshape(3)

    rvec = np.array(charuco["rvec"]).reshape(3)
    tvec = np.array(charuco["tvec"]).reshape(3)
    R_board2cam = cv2.Rodrigues(rvec)[0]
    t_board2cam = tvec

    R_gripper2base_all.append(R_base2ee.T)
    t_gripper2base_all.append(-R_base2ee.T @ t_base2ee)
    R_target2cam_all.append(R_board2cam)
    t_target2cam_all.append(t_board2cam)

## 3. RNASAC 기반 Tsai-Lenz 함수

In [19]:
def ransac_handeye(Rg2b_all, tg2b_all, Rt2c_all, tt2c_all, n_iter=100, sample_size=6, threshold=0.01):
    best_inliers = []
    best_R = None
    best_t = None
    n_total = len(Rg2b_all)

    for _ in range(n_iter):
        if n_total < sample_size:
            continue
        indices = random.sample(range(n_total), sample_size)
        Rg2b = [Rg2b_all[i] for i in indices]
        tg2b = [tg2b_all[i] for i in indices]
        Rt2c = [Rt2c_all[i] for i in indices]
        tt2c = [tt2c_all[i] for i in indices]

        try:
            R_est, t_est = cv2.calibrateHandEye(Rg2b, tg2b, Rt2c, tt2c, method=cv2.CALIB_HAND_EYE_TSAI)
        except:
            continue

        inliers = []
        for i in range(n_total):
            R_pred = Rg2b_all[i] @ R_est
            R_err = R_pred @ R_target2cam_all[i].T
            angle_error = np.linalg.norm(cv2.Rodrigues(R_err)[0])
            if angle_error < threshold:
                inliers.append(i)

        if len(inliers) > len(best_inliers):
            best_inliers = inliers
            best_R = R_est
            best_t = t_est

    return best_R, best_t, best_inliers

## 4.RANSAC 적용

In [20]:
sample_size = max(3, min(6, len(R_gripper2base_all)))

best_R, best_t, inliers = ransac_handeye(
    R_gripper2base_all, t_gripper2base_all,
    R_target2cam_all, t_target2cam_all,
    n_iter=200,
    sample_size=sample_size,
    threshold=0.01
)

# ==== 결과 저장 ====
if best_R is not None and best_t is not None:
    result = {
        "R_cam2ee": best_R.tolist(),
        "t_cam2ee": best_t.tolist(),
        "description": "RANSAC 기반 Tsai-Lenz (Camera → EE), 단위: meter"
    }
    with open("optimized_cam2ee_RANSAC.json", "w") as f:
        json.dump(result, f, indent=2)
    print("저장 완료: optimized_cam2ee_RANSAC.json")
else:
    print("RANSAC 실패: 적절한 해를 찾지 못함")

RANSAC 실패: 적절한 해를 찾지 못함


[ERROR:0@655.656] global calibration_handeye.cpp:335 calibrateHandEyeTsai Hand-eye calibration failed! Not enough informative motions--include larger rotations.
